# Train Binary Segmentation
Classes =  ['Background', 'UNKNOWN', 'General trash', 'Paper', 'Paper pack', 'Metal', 'Glass', 'Plastic', 'Styrofoam', 'Plastic bag', 'Battery', 'Clothing']

## Configures

In [1]:
import os
import sys

import torch
import numpy as np

import segmentation_models_pytorch as smp
from torch.optim import lr_scheduler

import Utils
from ModelManager import ModelManager
from DataManager import DataManager, CustomAugmentation
from TrainManager import TrainManager
from InferManager import InferManager
from LossManager import DiceLoss
from LearningRateManager import CustomCosineAnnealingWarmUpRestarts

Utils.fix_random_seed(random_seed=21)

project_dir = '/home/weebee/recyclables/baseline'
dataset_dir = os.path.join(project_dir, 'input')
save_dir = os.path.join(project_dir, 'saved/test_1')
if not os.path.isdir(dataset_dir):
    sys.exit('check dataset path!!')
if not os.path.isdir(save_dir):
    os.mkdir(save_dir)

batch_size = 5
number_worker = 4
epochs = 1
learning_rate = 1e-4

encoder = 'resnet101'
encoder_weights = 'imagenet'
activation = None
multi_gpu = False

transform = CustomAugmentation.to_tensor_transform()


In [2]:
print(Utils.get_classes())

# Make Model
model_manager = ModelManager()
model = model_manager.make_deeplabv3plus_model(
    encoder=encoder,
    encoder_weights=encoder_weights,
    class_number=2,
    activation=activation,
    multi_gpu=multi_gpu
)

criterion = smp.utils.losses.CrossEntropyLoss()
# criterion = smp.utils.losses.BCEWithLogitsLoss()
# criterion = tgm.losses.DiceLoss()
# criterion = DiceLoss()
# criterion = smp.utils.losses.JaccardLoss()

optimizer = torch.optim.Adam(
    [dict(params=model.parameters(),
          lr=learning_rate
          ),
     ])

lr_scheduler = None
# lr_scheduler = torch.optim.lr_scheduler.OneCycleLR(
#     optimizer, max_lr=0.01, steps_per_epoch=10, epochs=epochs, anneal_strategy='cos'
# )
# lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
#     optimizer, T_0=1, T_mult=2, eta_min=5e-5,
# )
# lr_scheduler = CustomCosineAnnealingWarmUpRestarts(
#     optimizer, T_0=20, T_mult=1, eta_max=0.1,  T_up=2, gamma=0.5
# )

data_manager = DataManager(dataset_path=dataset_dir)
train_manager = TrainManager()
infer_manager = InferManager()

['Background', 'UNKNOWN', 'General trash', 'Paper', 'Paper pack', 'Metal', 'Glass', 'Plastic', 'Styrofoam', 'Plastic bag', 'Battery', 'Clothing']


## Train each class

In [3]:
for target_class in Utils.get_classes():
    # Set Target class
    target_classes = ['Background']
    target_classes.append(target_class)

    # Load Dataset
    data_manager.assignDataLoaders(
        batch_size=batch_size,
        shuffle=True,
        number_worker=number_worker,
        drop_last=False,
        transform=transform,
        target_segmentation=True,
        target_classes=target_classes
    )

    # Run Train
    model = train_manager.run_train(
        model=model,
        num_epochs=epochs,
        data_loader=data_manager.train_data_loader,
        val_loader=data_manager.val_data_loader,
        criterion=criterion,
        optimizer=optimizer,
        lr_scheduler=lr_scheduler,
        save_dir=save_dir,
        file_name='best_model_target_' + '_'.join([v.lower() for v in target_classes[1:]]) + '.pt',
        class_number=len(target_classes),
        val_every=1
    )

    # Check Trained Model
    infer_manager.run_predict(
        model=model,
        data_loader=data_manager.val_data_loader,
        data_index_list=range(0, batch_size, int(batch_size/5)),  # data_index < batch_size
        save_dir=save_dir,
        class_name_list=target_classes,
        mode='val'
    )


loading annotations into memory...
Done (t=2.34s)
creating index...
index created!
loading annotations into memory...
Done (t=0.78s)
creating index...
index created!

Start training..
num_epochs: 1
save_dir: /home/weebee/recyclables/baseline/saved/test_1
file_name: best_model_target_paper.pt
val_every: 1



Epoch: 0/1, Time: 2021-12-27 14:15:07, lr: 0.0001: 100%|██████████| 524/524 [04:46<00:00,  1.83it/s, loss=0.206, mean_loss=0.202]



Start validation #1


Epoch: 1: 100%|██████████| 131/131 [00:23<00:00,  5.55it/s, mIoU=0.648, mIoU_list=0.757]


Validation #1  Average Loss: 0.1276, mIoU: 0.7572
Best performance at epoch: 1
Save model in /home/weebee/recyclables/baseline/saved/test_1
